In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import robustnessgym as rg

In [3]:
from spr.data.celeb import build_celeb_df

df = build_celeb_df.out(29).load()
dataset = rg.Dataset.load_image_dataset(
    df[df.split == "valid"].iloc[:128].to_dict("records"),    
    img_keys="img_path"
)

[2021-04-07 19:40:42,234][INFO][robustnessgym.core.dataformats.vision:178] :: Created `VisionDataset` with 128 rows and 45 columns.
[2021-04-07 19:40:42,237][INFO][root:237] :: Added column `index` with length `128`.


In [4]:
from spr.vision import train

model = train.get_artifacts("best_chkpt", run_id=201)["model"].load()

/dfs/scratch0/sabri/miniconda3/envs/rg/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [5]:
from robustnessgym.slicebuilders.discovery.bss import SeparatorSubpopulation



In [60]:
from robustnessgym.slicebuilders.discovery.bss import SeparatorSubpopulation

builder = SeparatorSubpopulation(
    model, 
    intervals=[('0%', '5%'), ('95%', '100%')],
    config={
        "activation_dim": 512,
        "target_module": "model.layer4",
        "num_components": 4,
        "pred_loss_weight": 100,
        "cov_loss_weight": 1e-06,
        "lr": 1e-1,
        "mixing_dropout_prob": 0.0,
        "cond_on_target": True
    }
)

In [61]:
out = builder(dataset, columns=["img_path", "male"], batch_size=16)

[2021-04-08 09:43:43,944][INFO][root:251] :: Removed column `comp_0`.
[2021-04-08 09:43:43,946][INFO][root:237] :: Added column `comp_0` with length `128`.
[2021-04-08 09:43:43,948][INFO][root:251] :: Removed column `comp_1`.
[2021-04-08 09:43:43,950][INFO][root:237] :: Added column `comp_1` with length `128`.
[2021-04-08 09:43:43,952][INFO][root:251] :: Removed column `comp_2`.
[2021-04-08 09:43:43,954][INFO][root:237] :: Added column `comp_2` with length `128`.
[2021-04-08 09:43:43,957][INFO][root:251] :: Removed column `comp_3`.
[2021-04-08 09:43:43,959][INFO][root:237] :: Added column `comp_3` with length `128`.

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
['32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63']

In [63]:
reduction_config = {
    "model": "none"
},
cluster_config = {
    "model": "kmeans",
    "metric_types": [
        "mean_loss",
        "composition"
    ],
    "search_k": True,
    "overcluster": True,
    "sil_cuda": True
}

In [65]:
from umap import UMAP
reduction_model = UMAP(n_components=2)
reducer = GEORGEReducer(reduction_config)
reducer.train(reduction_model, inputs=)

ModuleNotFoundError: No module named 'stratification'

In [66]:
import robustnessgym as rg 

# Load the boolq data
dataset = rg.Dataset.load_dataset('boolq', split='train[:10]')

[2021-04-08 10:42:49,264][WARNING][datasets.builder:357] :: Using custom data configuration default
[2021-04-08 10:42:49,272][WARNING][datasets.builder:512] :: Reusing dataset boolq (/afs/cs.stanford.edu/u/sabrieyuboglu/.cache/huggingface/datasets/boolq/default/0.1.0/1fcfdc6f36dc89a2245ffbbd5248ab33890594b50396731ebc78411bdd2ca534)
[2021-04-08 10:42:49,278][INFO][robustnessgym.core.dataformats.inmemory:87] :: Created `InMemoryDataset` with 10 rows and 3 columns.
[2021-04-08 10:42:49,281][INFO][root:133] :: Added column `index` with length `10`.


In [70]:
from robustnessgym import Spacy, Stanza, TextBlob

# Create the Spacy CachedOperation
spacy_op = Spacy()

# Apply it on the "text" column of a dataset
dataset = spacy_op(batch_or_dataset=dataset, columns=["question"])

# Easily retrieve whatever information you need, wherever you need it

# Retrieve the tokens extracted by Spacy for the first 2 examples in the dataset
tokens = Spacy.retrieve(batch=dataset[:2], columns=["question"], proc_fns=Spacy.tokens)

# Retrieve everything Spacy cached for the first 2 examples, and process it yourself
spacy_info = Spacy.retrieve(batch=dataset[:2], columns=["question"])

# ...do stuff with spacy_info

[2021-04-08 10:44:35,020][INFO][robustnessgym.core.dataformats.inmemory:281] :: Running update, a new dataset will be returned.
[2021-04-08 10:44:35,021][INFO][robustnessgym.core.dataformats.inmemory:87] :: Created `InMemoryDataset` with 0 rows and 0 columns.
[2021-04-08 10:44:35,022][INFO][robustnessgym.core.dataformats.inmemory:426] :: Running `map`, the dataset will be left unchanged.


[2021-04-08 10:44:35,067][INFO][root:133] :: Added column `Spacy(lang=en_core_web_sm, neuralcoref=False, columns=['question'])` with length `10`.



In [71]:
dataset.column_names

['answer',
 'passage',
 'question',
 'index',
 "Spacy(lang=en_core_web_sm, neuralcoref=False, columns=['question'])"]

In [14]:
from spr.data.celeb import build_celeb_df, celeb_transform

df = build_celeb_df.out(29).load()
dataset = rg.Dataset.load_image_dataset(
    df[df.split == "valid"].iloc[:128].to_dict("records"),    
    img_columns="img_path",
    transform=celeb_transform
)

[2021-04-08 22:44:15,321][INFO][robustnessgym.core.dataformats.vision:247] :: Created `VisionDataset` with 128 rows and 45 columns.
[2021-04-08 22:44:15,324][INFO][root:306] :: Added column `index` with length `128`.


In [15]:
from spr.vision import train
from robustnessgym.cachedops.activation import ActivationCachedOp
model = train.get_artifacts("best_chkpt", run_id=201)["model"].load()
act_op = ActivationCachedOp(
    model=model, target_module="model.layer4", device=0
)

In [16]:
out = act_op(dataset, columns=["img_path"], batch_size=4)

[2021-04-08 22:44:16,474][INFO][robustnessgym.core.dataformats.vision:521] :: Running update, a new dataset will be returned.
[2021-04-08 22:44:16,475][INFO][robustnessgym.core.dataformats.vision:247] :: Created `VisionDataset` with 0 rows and 0 columns.
[2021-04-08 22:44:20,425][INFO][robustnessgym.core.dataformats.vision:721] :: Running `map`, the dataset will be left unchanged.



[2021-04-08 22:44:21,134][INFO][root:306] :: Added column `ActivationCachedOp(columns=['img_path'])` with length `128`.


In [17]:
act = ActivationCachedOp.retrieve(out[:3], columns=["img_path"])

In [22]:
list(act[0].shape)

[512, 8, 8]

In [41]:
batch = dataset[:100]

In [42]:
img = batch["img_path"]

here


In [43]:
img.shape

torch.Size([100, 3, 256, 256])

In [8]:
from torch.nn import Linear
lin = Linear(5,6)

In [12]:
lin._get_name()

'Linear'

In [15]:
model.model.layer4

'Sequential'